<a href="https://colab.research.google.com/github/mahendravala/Deep-learning-AI-00111-00111D/blob/master/My_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/My Docs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 cnn_architectures
'eDocket Status_09Oct19.gsheet'
 Felix
 IMG_1935.JPG
 Machine-Learning
 MS_Dataset_2019.rar
 my-docs.pdf
'New Doc 2017-07-17.pdf'
 themeforest-22893876-indobiz-factory-industrial-html-template.zip
'Untitled Diagram.html'


In [3]:
!ls "/content/drive/My Drive/My Docs/cnn_architectures"


test  train


In [0]:
IMAGE_SIZE = [224, 224]
target_size = (224,224)
batchsize=[2, 4, 8, 10]

In [0]:
#Give dataset path
train_path = '/content/drive/My Drive/My Docs/cnn_architectures/train'
test_path = '/content/drive/My Drive/My Docs/cnn_architectures/test'

In [0]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [0]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [8]:
# useful for getting number of classes
folders = glob('/content/drive/My Drive/My Docs/cnn_architectures/train/*')
print(len(folders))

4


In [9]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
#adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [13]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory(train_path,
                                              target_size = (224, 224),
                                              batch_size = 32,
                                              class_mode = 'categorical')

Found 34 images belonging to 4 classes.


In [14]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 8 images belonging to 4 classes.


In [15]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np

lr_scheduler = LearningRateScheduler(0.01, verbose=1)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

num_epochs = 1000
num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/My Docs/cnn_architectures/my_vgg_model.h5',
                             verbose=1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=32,
  callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/10
5/5 [==============================] - 44s 9s/step - loss: 4.4599 - accuracy: 0.6786 - val_loss: 4.7928 - val_accuracy: 0.7344

Epoch 00001: val_loss improved from inf to 4.79284, saving model to /content/drive/My Drive/My Docs/cnn_architectures/my_vgg_model.h5
Epoch 2/10
5/5 [==============================] - 25s 5s/step - loss: 3.3141 - accuracy: 0.7400 - val_loss: 2.8757 - val_accuracy: 0.7578

Epoch 00002: val_loss improved from 4.79284 to 2.87571, saving model to /content/drive/My Drive/My Docs/cnn_architectures/my_vgg_model.h5
Epoch 3/10
5/5 [==============================] - 22s 4s/step - loss: 3.1798 - accuracy: 0.7571 - val_loss: 3.8343 - val_accuracy: 0.7539

Epoch 00003: val_loss did not improve from 2.87571
Epoch 4/10
5/5 [==============================] - 24s 5s/step - loss: 2.5572 - accuracy: 0.7550 - val_loss: 3.8343 - val_accuracy: 0.7422

Epoch 00004: val_loss did not improve from 2.87571
Epoch 5/10
5/5 [==============================] - 23s 5s/step - loss: 

In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [3]:
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/My Docs"

img = test_path = '/content/drive/My Drive/My Docs/cnn_architectures/0.jpeg'

model1 = tf.keras.models.load_model('/content/drive/My Drive/My Docs/cnn_architectures/my_vgg_model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 cnn_architectures
'eDocket Status_09Oct19.gsheet'
 Felix
 IMG_1935.JPG
 Machine-Learning
 MS_Dataset_2019.rar
 my-docs.pdf
'New Doc 2017-07-17.pdf'
 themeforest-22893876-indobiz-factory-industrial-html-template.zip
'Untitled Diagram.html'


In [0]:
import cv2
import numpy as np
from keras.preprocessing import image
img = cv2.imread('/content/drive/My Drive/My Docs/cnn_architectures/0.jpeg')
#cv2.show()

dim = (32, 32)
img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)
preds = model1.predict(x)